In [1]:
import random

## Parameters

In [2]:
#
# Use e.g. https://www.compilejava.net/
#
#import java.util.*;
#import java.math.*;
#
#public class Entrypoint
#{
#  public static void main(String[] args)
#  {
#    BigInteger q = BigInteger.probablePrime(128, new Random());    
#    System.out.println(q);
#  }
#}

# small and large field
Q = 6497992661811505123 # < 64 bits
P = 1802216888453791673313287943102424579859887305661122324585863735744776691801009887 # < 270 bits

## Sharing and reconstruction

In [3]:
def share(secret, field=Q):
    share0 = random.randrange(field)
    share1 = (secret - share0) % field
    return [share0, share1]

def reconstruct(shares, field=Q):
    return sum(shares) % field

In [4]:
x = share(5)
print(x)
assert reconstruct(x) == 5

[800583186357053302, 5697409475454451826]


## Linear operations

In [5]:
def add(x, y, field=Q):
    z0 = (x[0] + y[0]) % field
    z1 = (x[1] + y[1]) % field
    return [z0, z1]

def sub(x, y, field=Q):
    z0 = (x[0] - y[0]) % field
    z1 = (x[1] - y[1]) % field
    return [z0, z1]

x = share(5)
y = share(3)
assert reconstruct(add(x, y)) == 8
assert reconstruct(sub(x, y)) == 2

In [6]:
def add_public(x, k, field=Q):
    y0 = (x[0] + k) % field
    y1 =  x[1]
    return [y0, y1]

def sub_public(x, k, field=Q):
    y0 = (x[0] - k) % field
    y1 =  x[1]
    return [y0, y1]

def mul_public(x, k, field=Q):
    y0 = (x[0] * k) % field
    y1 = (x[1] * k) % field
    return [y0, y1]

x = share(5)
k = 2
y = add_public(x, k); assert reconstruct(y) == 7
y = sub_public(x, k); assert reconstruct(y) == 3
y = mul_public(x, k); assert reconstruct(y) == 10

## Multiplication

In [7]:
def generate_mul_triple(field=Q):
    a = random.randrange(field)
    b = random.randrange(field)
    c = a * b % field
    return share(a, field), share(b, field), share(c, field)

(a, b, c) = generate_mul_triple()
assert reconstruct(a) * reconstruct(b) % Q == reconstruct(c)

In [8]:
def mul(x, y, triple, field=Q):
    a, b, c = triple
    # local masking
    d = sub(x, a, field)
    e = sub(y, b, field)
    # communication: the players simultanously send their shares to the other
    alpha = reconstruct(d, field)
    beta  = reconstruct(e, field)
    # local combination
    f = alpha * beta % field
    g = mul_public(a, beta, field)
    h = mul_public(b, alpha, field)
    return add(h, add(g, add_public(c, f, field), field), field)

x = share(5)
y = share(2)
z = mul(x, y, generate_mul_triple()); assert reconstruct(z) == 10

## Powering and polynomials

In [10]:
def generate_powering_triple(exponent, field=Q):
    a = random.randrange(field)
    return [ share(pow(a, e, field)) for e in range(1, exponent+1) ]

a, aa, aaa, aaaa = generate_powering_triple(4)
assert reconstruct(a) * reconstruct(a) % Q == reconstruct(aa) 
assert reconstruct(aa) * reconstruct(a) % Q == reconstruct(aaa)
assert reconstruct(aaa) * reconstruct(a) % Q == reconstruct(aaaa)

In [11]:
from functools import reduce
from scipy.misc import comb
binom = lambda n, k: comb(n, k, exact=True)

ONE = [1,0] # constant sharing of 1

def pows(x, triple, field=Q):
    # local masking
    a = triple[0]
    v = sub(x, a)
    # communication: the players simultanously send their share to the other
    epsilon = reconstruct(v)
    # local combination
    x_powers = []
    for exponent in range(1, len(triple)+1):
        # prepare all term values
        a_powers = [ONE] + triple[:exponent]
        e_powers = [ pow(epsilon, e, Q) for e in range(exponent+1) ]
        coeffs   = [ binom(exponent, k) for k in range(exponent+1) ]
        # compute and sum terms
        terms = ( mul_public(a, c * e) for a, c, e in zip(a_powers, coeffs, reversed(e_powers)) )
        x_powers.append(reduce(lambda x,y: add(x, y, field), terms))
    return x_powers

x = share(4)

xs = pows(x, generate_powering_triple(3))
assert [ reconstruct(x) for x in xs ] == [ pow(4,e,Q) for e in range(1,3+1) ]

xs = pows(x, generate_powering_triple(10))
assert [ reconstruct(x) for x in xs ] == [ pow(4,e,Q) for e in range(1,10+1) ]

ModuleNotFoundError: No module named 'scipy'

In [ ]:
def pol_public(x, coeffs, triple, field):
    powers = [ONE] + pows(x, triple, field)
    terms = ( mul_public(xe, ce, field) for xe,ce in zip(powers, coeffs) )
    return reduce(lambda y,z: add(y, z, field), terms)

x = share(5)
coeffs = [1,2,3,4]
y = pol_public(x, coeffs, generate_powering_triple(len(coeffs)-1, Q), Q)
assert reconstruct(y) == (1*pow(5,0) + 2*pow(5,1) + 3*pow(5,2) + 4*pow(5,3)) % Q

### Up and down sharing

In [ ]:
def generate_statistical_mask():
    return random.randrange(2*BOUND * BASE**KAPPA)

def generate_zero_triple(field):
    return share(0, field)

def convert(x, from_field, to_field, zero_triple):
    # local mapping to positive representation
    x = add_public(x, BOUND, from_field)
    # local masking and conversion by player 0
    r = generate_statistical_mask()
    y0 = (zero_triple[0] - r) % to_field
    # exchange of masked share: one round of communication
    e = (x[0] + r) % from_field
    # local conversion by player 1
    xr = (e + x[1]) % from_field
    y1 = (zero_triple[1] + xr) % to_field
    # local combine
    y = [y0, y1]
    # local mapping back from positive representation
    y = sub_public(y, BOUND, to_field)
    return y

def upshare(x, large_zero_triple):
    return convert(x, Q, P, large_zero_triple)

def downshare(x, small_zero_triple):
    return convert(x, P, Q, small_zero_triple)

x = share(encode(-.5, Q), Q)
y = upshare(x, generate_zero_triple(P))
assert reconstruct(y, P) == encode(-.5, P)
z = downshare(y, generate_zero_triple(Q))
assert reconstruct(z, Q) == encode(-.5, Q)

In [ ]:
x = share(encode(-.5, Q), Q)
x2 = mul(x, x, generate_multiplication_triple(Q), Q)
x4 = mul(x2, x2, generate_multiplication_triple(Q), Q)
y = truncate(x4, 4 * PRECISION_FRACTIONAL - PRECISION_FRACTIONAL, Q)
assert not decode(reconstruct(y, Q), Q) == (-.5)**4

x = share(encode(-.5, Q), Q)
x = upshare(x, generate_zero_triple(P))
x2 = mul(x, x, generate_multiplication_triple(P), P)
x4 = mul(x2, x2, generate_multiplication_triple(P), P)
y = truncate(x4, 4 * PRECISION_FRACTIONAL - PRECISION_FRACTIONAL, P)
y = downshare(y, generate_zero_triple(Q))
assert decode(reconstruct(y, Q), Q) == (-.5)**4

## Fixed-point encoding

In [12]:

BASE = 10
KAPPA = 9 # ~29 bits

PRECISION_INTEGRAL = 2
PRECISION_FRACTIONAL = 7
PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
BOUND = BASE**PRECISION

Q_MAXDEGREE = 2
assert Q > BASE**(PRECISION * Q_MAXDEGREE) # supported multiplication degree (without truncation)
assert Q > 2*BOUND * BASE**KAPPA # supported kappa when in positive range 

P_MAXDEGREE = 9
assert P > Q
assert P > BASE**(PRECISION * P_MAXDEGREE)

In [13]:
def encode(rational, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = int(rational * BASE**precision_fractional)
    field_element = upscaled % field
    return field_element

def decode(field_element, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = field_element if field_element <= field/2 else field_element - field
    rational = upscaled / BASE**precision_fractional
    return rational

In [14]:
# using trick from SecureML paper that only requires a local operation

def truncate(x, amount=PRECISION_FRACTIONAL, field=Q):
    y0 = x[0] // BASE**amount
    y1 = field - ((field - x[1]) // BASE**amount)
    return [y0, y1]

In [16]:
x = share(encode(-.5))

x2 = truncate(mul( x,  x, generate_mul_triple()), PRECISION_FRACTIONAL)
x4 = truncate(mul(x2, x2, generate_mul_triple()), PRECISION_FRACTIONAL)
y = decode(reconstruct(x4))
assert y == (-.5)**4

x2 = mul( x,  x, generate_mul_triple())
x4 = mul(x2, x2, generate_mul_triple())
y = decode(reconstruct(truncate(x4, 2*PRECISION_FRACTIONAL)))
assert not y == (-.5)**4